## One round federated learning simulation

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


### Config variables for models and training


In [3]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_local_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True


### Load data


In [4]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


Files already downloaded and verified
Files already downloaded and verified


### Create two local models


In [ ]:

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)



### Train first local model


In [ ]:
model1_name = 'fl_model1'

# Training
model.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=reg)

train(model1, modelpath, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



### Train second local model

In [ ]:
model2_name = 'fl_model2'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, modelpath, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

### Test the trained models

In [ ]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model1_name, '.ckpt'))
model.load_state_dict(model_ckpt)
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model2_name, '.ckpt'))
model.load_state_dict(model_ckpt)
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))


### Aggregate the local models


In [ ]:
from client.utils import weights_zero_init
from client.aggregators import federated_aggregate
from os.path import join as pjoin


model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
# print(model3)

model_avg = federated_aggregate([model1, model3])

# model_avg = federated_aggregate([model1, model2])


for parameter in model1.parameters():
    print(parameter)
  
# for parameter in model2.parameters():
#     print(parameter)  

for parameter in model_avg.parameters():
    print(parameter)
    


### Print the new global model


In [ ]:
for parameter in model1.parameters():
    print(parameter)
  
# for parameter in model2.parameters():
#     print(parameter)  

for parameter in model_avg.parameters():
    print(parameter)

### Test the new global model

In [ ]:

model = model_avg
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))